In [13]:
import pandas as pd
from os import chdir
import subprocess
import glob

# local import (Hirotaka's myfunctions.py script)
import myfunctions as myfunc

In [2]:
!mkdir temp
chdir('temp')

mkdir: cannot create directory 'temp': File exists


In [3]:
# Get the id file, covariate file (all European)
wgsfolder = '/data/CARD/PD/GENOMES/august19/genotypes'
idfile = '_keep.id'
demog = pd.read_csv('/data/LNG/iwakih2/ampPD/data1_out/PP_PD_BF_HB_LC_demog.csv')
euro = pd.read_csv(f'{wgsfolder}/PCA_filtered_europeans.txt', sep=' ',
                  header=None, names=['FID', 'IID'])
df = pd.merge(demog, euro, left_on='WGSID', right_on='IID')
df['FID'] = df.WGSID
df['IID'] = df.WGSID
df.RECRUIT=['CTR' if i=='HC' else i for i in df.RECRUIT]
df = df[df.RECRUIT.isin(['CTR', 'PD'])]
df = df[df.RECRUIT==df.LASTDIAG]
df['pheno'] = [1 if i=='PD' else 0 for i in df.RECRUIT]
df['AAD'] = df.AADi.where(df.pheno==0, other=df.AAOi)
df['AGE'] = df.AGEatBL.where(df.pheno==0, other=df.AAD)

df[['FID', 'IID', 'pheno']].to_csv(idfile, index=False, sep='\t', header=None)
df[['FID', 'IID', 'AGE', 'FEMALE']].to_csv('_qcovar.txt', index=False, sep='\t', header=None)

In [ ]:
df.head()

In [ ]:
# for chrnum in range(1,23):
#     pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
#     out = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#     plinkcmd = f'\
#  plink2 --pfile {pfile}\
#  --make-bed\
#  --out {out}'
#    myfunc.shell_do(plinkcmd)

In [ ]:
# with open('_mergelist.txt', 'w') as f:
#     for chrnum in range(1,23):
#         bfile = f'{wgsfolder}/merge/pd.june2019.chr{chrnum}.sqc'
#         f.write(f'{bfile}\n')
    

In [ ]:
# mergecmd = f'\
# plink --merge-list _mergelist.txt\
#  --make-bed\
#  --out {wgsfolder}/merge/pd.june2019.sqc'
# with open('_merge_batch_script.sh', 'w') as f:
#     f.write('#!/bin/bash\n')
#     f.write('module load plink\n')
#     f.write(f'{mergecmd}')

In [ ]:
# !sbatch _merge_batch_script.sh --cpus-per-task=128 --mem=100g

In [ ]:
# used this to generate pca for chr1 just for testing purposes-- prune and merge all chrs first then run PCA
with open('_pca.swarm', 'w') as f:
    for chrnum in range(1,2):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        out_pcs = f'chr{chrnum}.pca'
        plink_pca_cmd = f'\
plink2 --pfile {pfile}\
 --pca 50\
 --out {out_pcs}\n'
        
        f.write(plink_pca_cmd)
        print(plink_pca_cmd)

In [ ]:
# !swarm -f _pca.swarm -g 64 --time=10:00:00 -t 20 --logdir swarm --module plink/2.0_alpha_1_final --partition=norm

In [ ]:
with open('_script1.sh', 'w') as f:
    for chrnum in range(1,):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        plinkcmd = f"\
plink2 --pfile {pfile}\
 --keep {idfile} \
 --freq \
 --out {freqfile}\n"
        f.write(plinkcmd)
#         myfunc.shell_do(plinkcmd)
print(f'{plinkcmd}')

In [33]:
from dask.distributed import Client, progress
# client should match the qued number of threads
# client = Client(processes=False, threads_per_worker=2,
#                 n_workers=2, memory_limit='30GB')
# import dask
import dask.dataframe as dd

#try testing fewer maf bins
def MAFbin(x):
    if x>0:
        if x<0.001:
            return(1)
        elif x<0.01:
            return(2)
        elif x<0.1:
            return(3)
        elif x<0.25:
            return(4)
        elif x<0.5:
            return(5)
        else:
            return(6)
    else:
        return(0)



# def MAFbin(x):
#     if x>0:
#         if x<0.001:
#             return(1)
#         elif x<0.01:
#             return(2)
#         elif x<0.1:
#             return(3)
#         elif x<0.2:
#             return(4)
#         elif x<0.3:
#             return(5)
#         elif x<0.4:
#             return(6)
#         else:
#             return(7)
#     else:
#         return(0)



In [ ]:
client.close()

In [ ]:
total_counts_df = pd.DataFrame(index=[0,1,2,3,4,5,6])

for chrnum in range(1,23):
    freqfile = f'freq.{chrnum}'

    df = dd.read_csv(f'{freqfile}.afreq', sep='\t')
    df['MAF'] = df.ALT_FREQS.where(cond=df.ALT_FREQS<=0.5, other=1-df.ALT_FREQS)
    df['Group'] = df.MAF.apply(MAFbin, meta=('MAF', 'float64'))
    for i in range(1,7):
        df.loc[df.Group==i, ['ID']].to_csv(
            f'{freqfile}.mafbin{i}', index=False, single_file=True)
    print(f'chr{chrnum} finished')
    total_counts_df[chrnum] = pd.DataFrame(df.groupby('Group').ID.count().compute())
#     total_counts_df.merge(counts_df, left_index=True)

In [ ]:
# # add chr names
chrs = ['chr' + str(i) for i in range(1,23)]
total_counts_df.columns = chrs

# add bins
bins = ['0.000', '<0.001', '<0.01', '<0.1', '<0.25', '<0.5', '=0.5']
total_counts_df['bin'] = bins
total_counts_df.set_index('bin', inplace=True)
total_counts_df['total'] = total_counts_df.sum(axis=1)
total_counts_df.to_csv('_maf_bin_counts.txt', sep='\t')
total_counts_df

In [ ]:
freqfile = f'freq.21'

df = pd.read_csv(f'{freqfile}.afreq', sep='\t')
df['MAF'] = df.ALT_FREQS.where(cond=df.ALT_FREQS<=0.5, other=1-df.ALT_FREQS)


In [ ]:
df['Group'] = df.MAF.apply(MAFbin)
df.head()

In [ ]:
df[df.Group == 6].head()

In [ ]:
with open('_script2.swarm', 'w') as f:
    for chrnum in range(1,23):
        pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
        freqfile = f'freq.{chrnum}'
        for Indx in range(1,7):
            mafbinIndx = f'mafbin{Indx}'
            mafbinfile =  f'{freqfile}.{mafbinIndx}'

            plinkcmd = f"\
plink2 --pfile {pfile}\
 --extract {mafbinfile}\
 --keep {idfile}\
 --make-bed\
 --out {mafbinfile}"
            gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin\
 --thread-num 10'

            f.write(f'{plinkcmd} && {gctacmd1}\n')

In [ ]:
# !swarm -f _script2.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA,plink/2.0_alpha_1_final --partition=norm --devel

In [ ]:
for chrnum in range(1,23):
    pass

In [ ]:
# need to rerun for jobs that were killed
outputs = glob.glob('freq.*.mafbin*.ldbin.score.ld')
# len(glob.glob('*.ldbin.log'))
expected_out = ['freq.' + str(i) + '.mafbin' + str(j) + '.ldbin.score.ld' for i in range(1,23) for j in range(1,7)]
missing = [bed.replace('.ldbin.score.ld', '') for bed in list(set(expected_out) - set(outputs))]

with open('_rerunscript.swarm', 'w') as f:
    for mafbinfile in missing:
        gctacmd1 = f'\
gcta64 --bfile {mafbinfile}\
 --ld-score-region 10000\
 --out {mafbinfile}.ldbin\
 --thread-num 10'

        f.write(f'{gctacmd1}\n')

In [ ]:
!swarm -f _rerunscript.swarm --time=40:00:00 -g 64 -t 10 --logdir swarm --module GCTA --partition=norm

In [ ]:
for chrnum in range(1,23):
    pfile = f'{wgsfolder}/pd.june2019.chr{chrnum}.sqc'
    freqfile = f'freq.{chrnum}'
    for Indx in range(1,7):
        mafbinIndx = f'mafbin{Indx}'
        mafbinfile =  f'{freqfile}.{mafbinIndx}'

        
        

        # Note this is the ldscore within the maf bin
        df = dd.read_csv(f'{mafbinfile}.ldbin.score.ld', sep=' ')
        ldscvec = df.ldscore_SNP.compute()
        ldscThres = ldscvec.median()
        df['Group'] = 1
        df['Group'] = df.Group.where(cond=df.ldscore_SNP<ldscThres, other=2)


        # Create ldbin and GRMs within the ldbin
        for i in range(1, 3):
            ldbinIndx = f'{mafbinfile}.ldbin{i}'
#             df.loc[df.Group==i, ['SNP']].to_csv(
#                 f'{ldbinIndx}', index=False, single_file=True)
#             gctacmd2 = f'\
#             gcta64 --bfile {mafbinfile}\
#  --extract {ldbinIndx}\
#  --make-grm\
#  --out {ldbinIndx}'
#             myfunc.shell_do(gctacmd2)
            with open('multi_GRMs.txt', 'a') as f:
                f.write(f'{ldbinIndx}\n')
                print(ldbinIndx)
        # mafbin information save to mafbinInfo.csv
        with open('mafbinInfo.txt', 'a') as f:
            s = f'{chrnum}\t{mafbinIndx}\t{len(ldscvec)}\t{ldscvec.mean()}\t{ldscvec.median()}\t{ldscvec.std()}'
            f.write(f'{s}\n')

In [ ]:
# create covar file per chrom and impute missing age (may want to consider dropping age column in this case (missing for more than half))
# impute "FEMALE" with median
for chrnum in range(1,2):
    pcs_df = pd.read_csv(f'chr{chrnum}.pca.eigenvec', sep='\t')
    pcs_df.rename(columns={'#FID':'FID'}, inplace=True)
    covar_df = df[['FID', 'IID', 'AGE', 'FEMALE']].merge(pcs_df, on=['FID','IID'])
    covar_df.drop(['AGE','FEMALE'], axis=1, inplace=True)
#     covar_df.AGE.fillna(covar_df.AGE.mean(), inplace=True)
#     covar_df.FEMALE.fillna(covar_df.FEMALE.median(),inplace=True)
#     print(f'dropping {covar_df[covar_df.AGE.isnull()].shape[0]} rows for missing age')
#     covar_df.dropna(how='any', inplace=True)
    covar_df.to_csv(f'chr{chrnum}.covar', index=False, sep=' ', header=None)
    print(f'wrote: chr{chrnum}.covar with {covar_df.shape[0]} samples')

In [ ]:
covar_df.head()

In [57]:
high_ld_counts_dict = {i:0 for i in range(1,7)}
low_ld_counts_dict = {i:0 for i in range(1,7)}

for chrnum in range(1,23):
    freqfile = f'freq.{chrnum}'
    for Indx in range(1,7):
        mafbinIndx = f'mafbin{Indx}'
        mafbinfile =  f'{freqfile}.{mafbinIndx}'
        for i in range(1, 3):
            ldbinIndx = f'{mafbinfile}.ldbin{i}'
            ldbin_df = pd.read_csv(ldbinIndx,sep='\t')
            if (i == 1):
                high_ld_counts_dict[Indx] += ldbin_df.shape[0]
            else:
                low_ld_counts_dict[Indx] += ldbin_df.shape[0]
       

In [86]:
print(f'high ld counts per maf group: {high_ld_counts_dict}')
print(f'low ld counts per maf group: {low_ld_counts_dict}')
high_df = pd.Series(high_ld_counts_dict).to_frame()
low_df = pd.Series(low_ld_counts_dict).to_frame()
high_df.columns = ['high_ld']
low_df.columns = ['low_ld']
ld_counts_df = high_df.merge(low_df, left_index=True, right_index=True)
bins = ['<0.001', '<0.01', '<0.1', '<0.25', '<0.5', '=0.5']
ld_counts_df['maf_bin'] = bins
ld_counts_df.set_index('maf_bin', inplace=True)
ld_counts_df.to_csv('_ld_bin_counts.txt', sep='\t')
ld_counts_df

high ld counts per maf group: {1: 31016882, 2: 5946277, 3: 3372240, 4: 1653495, 5: 1833686, 6: 1108}
low ld counts per maf group: {1: 31017845, 2: 5946340, 3: 3372272, 4: 1653513, 5: 1833710, 6: 1273}


,high_ld,low_ld
maf_bin,,
<0.001,31016882,31017845
<0.01,5946277,5946340
<0.1,3372240,3372272
<0.25,1653495,1653513
<0.5,1833686,1833710
=0.5,1108,1273


In [8]:
#--reml-no-lrt\
# --reml-inv-mtd 3\

gctacmd3 = f'\
gcta --reml\
 --mgrm multi_GRMs.txt\
 --pheno {idfile}\
 --qcovar chr1.pca.eigenvec\
 --reml-alg 2\
 --reml-no-constrain\
 --thread-num 60\
 --reml-maxit 1000\
 --out test'
# myfunc.shell_do(gctacmd3)

with open('_reml_script.swarm', 'w') as f:
#     f.write('#!/bin/bash\n')
    f.write(f'{gctacmd3}\n')

In [30]:
!swarm -f _reml_script.swarm --time=40:00:00 -g 64 -t 64 --logdir swarm --module GCTA --partition=norm

61143262


In [47]:
test_df = pd.read_csv('freq.22.mafbin1',sep='\t')

In [49]:
test_df.shape[0]

973767

In [16]:
# get estimates for cpu-hrs for analysis
jobs_df = pd.read_csv('../biowulf_user_dashboard.csv')
jobs_df.head(10)

,jobid,jobname,state,statetime,nodelist,eval,user,submit_time,start_time,end_time,...,std_in,std_out,std_err,comment,jobidarray,jobidraw,submit_time_human,start_time_human,end_time_human,Unnamed: 47
0,61136072_0 Biowul...,swarm,RUNNING,1594656203,cn1088,NaN,vitaled2,1594656136,1594656203,0,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,61136072_0,61136072,0x1,2020-07-13 12:02:16 EDT,2020-07-13 12:03:23 EDT,NaN
1,61132199_0 Biowul...,swarm,RUNNING,1594651119,cn3406,NaN,vitaled2,1594651043,1594651119,0,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,61132199_0,61132199,0x1,2020-07-13 10:37:23 EDT,2020-07-13 10:38:39 EDT,NaN
2,61128454 Biowulf ...,sinteractive,RUNNING,1594646752,cn4244,NaN,vitaled2,1594646750,1594646752,0,...,NaN,NaN,NaN,NaN,NaN,61128454,NaN,2020-07-13 09:25:50 EDT,2020-07-13 09:25:52 EDT,NaN
3,61074473 Biowulf ...,sinteractive,TIMEOUT,1594612409,cn0855,NaN,vitaled2,1594565577,1594565580,1594612409,...,NaN,NaN,NaN,NaN,NaN,61074473,NaN,2020-07-12 10:52:57 EDT,2020-07-12 10:53:00 EDT,2020-07-12 23:53:29 EDT
4,61078515 Biowulf ...,_reml_script.sh,FAILED,1594572629,cn0861,NaN,vitaled2,1594572624,1594572628,1594572629,...,NaN,NaN,NaN,NaN,NaN,61078515,NaN,2020-07-12 12:50:24 EDT,2020-07-12 12:50:28 EDT,2020-07-12 12:50:29 EDT
5,60967285 Biowulf ...,sinteractive,TIMEOUT,1594458695,cn0927,NaN,vitaled2,1594411836,1594411887,1594458695,...,NaN,NaN,NaN,NaN,NaN,60967285,NaN,2020-07-10 16:10:36 EDT,2020-07-10 16:11:27 EDT,2020-07-11 05:11:35 EDT
6,60892130_4 Biowul...,swarm,COMPLETED,1594397848,cn1948,NaN,vitaled2,1594320771,1594320911,1594397848,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,60892130_4,60892356,NaN,2020-07-09 14:52:51 EDT,2020-07-09 14:55:11 EDT,2020-07-10 12:17:28 EDT
7,60892130_3 Biowul...,swarm,COMPLETED,1594389998,cn1931,NaN,vitaled2,1594320771,1594320911,1594389998,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,60892130_3,60892355,NaN,2020-07-09 14:52:51 EDT,2020-07-09 14:55:11 EDT,2020-07-10 10:06:38 EDT
8,60892130_0 Biowul...,swarm,COMPLETED,1594368019,cn1679,NaN,vitaled2,1594320771,1594320910,1594368019,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,60892130_0,60892352,NaN,2020-07-09 14:52:51 EDT,2020-07-09 14:55:10 EDT,2020-07-10 04:00:19 EDT
9,60892130_6 Biowul...,swarm,COMPLETED,1594367315,cn1987,NaN,vitaled2,1594320771,1594320911,1594367315,...,/dev/null,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,/gpfs/gsfs12/users/vitaled2/rare_variants/temp...,NaN,60892130_6,60892130,0x7F,2020-07-09 14:52:51 EDT,2020-07-09 14:55:11 EDT,2020-07-10 03:48:35 EDT


In [27]:
jobs_df2 = jobs_df[jobs_df.state == 'COMPLETED'][['jobid','cpu', 'cpu_min',
       'cpu_max', 'cpu_avg', 'cpu_util', 'mem', 'mem_min', 'mem_max','elapsed_time']]

jobs_df2['hrs'] = jobs_df2.elapsed_time/3600
jobs_df2['cpu_hrs'] = jobs_df2.cpu * jobs_df2.hrs

In [29]:
sum(jobs_df2.cpu_hrs)

6078.294999999996